In [1]:
import sys
sys.path.append('..')
import theano.tensor as T
import theano
import numpy
import theano_lstm
import numpy as np
import data_managers
import os

# Plotting
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

In [2]:
net = theano_lstm.lstm_rnn( 300, [(128,128)], 150, save_weights_every=50000)

//anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [3]:
%%time
net.initialize_training_adam()

CPU times: user 2min 33s, sys: 2.31 s, total: 2min 35s
Wall time: 2min 38s


In [21]:
reload(data_managers)
data_butler = data_managers.character_data_manager('/Users/alex/Desktop/data manager/',
                                                   batch_size=1,perms_per=5,minmax_doc_length=[0,300])
data_butler.convert_ent_to_one_hot()

Now offering: Training data!


In [22]:
def smooth_rep(data,sigma):
    w = np.ceil(2*sigma)
    f = np.exp( -.5*(np.arange(-w,w)**2)/(sigma**2) )
    f = f / np.sum(f)

    y = np.convolve(data,f,'valid')
    x = np.arange(w,len(data)-w+1)
    return x, y

In [31]:
# Re-initialize the network
net.initialize_network_weights(scale_down=0.6, b_f_offset=0.75)

# Initialize cost 
cost = []

data_butler.set_batch_size(1)
data_butler.set_perms_per(5)

Batch Size = 1;  Stride = 1
5 examples per offer: Batch Size = 1  *  Permutations per = 5


In [ ]:
n_batches = 200

start_plot_at = 0
smooth_sigma = 25

#Create plot 
fig = plt.figure(figsize=(18,30))
axB = fig.add_subplot(321)
axS = fig.add_subplot(322)
fig.subplots_adjust(wspace=.2)

labels = ['Input gate','Forget gate','memory Cell','Output gate','Hidden activation']
ax_list = []
for sp in range(len(labels)):
    ax_list.append(fig.add_subplot(3,len(labels),(1*len(labels))+sp+1))

axD = fig.add_subplot(3,1,3)

for batch in range(n_batches):
    
    # Get the permuted data
    vectors, lengths, answer = data_butler.offer_data()
    
    # Step 
    temp_cost,_ = net.adam_step(vectors, lengths, answer)
    cost.append(temp_cost)
    
    # Update plot
    if len(cost) > start_plot_at:
        C = np.array(cost[start_plot_at:])
    else:
        C = cost
    x, y = smooth_rep( C, smooth_sigma )
    axB.cla();
    axS.cla();
    if len(x) == len(y):
        axB.plot( np.arange(start_plot_at,len(cost)), C, 'bo', x+start_plot_at, y, 'r')
        axS.plot( x+start_plot_at, y, 'r')
    else:
        axB.plot( np.arange(start_plot_at,len(cost)), C, 'bo' )

    hidden_players = net.hidden_activations(vectors[:,:,:2])
    for ax,gtp,title in zip(ax_list, hidden_players, labels):
        ax.imshow(gtp[:lengths[1],:,1]-gtp[:lengths[0],:,0])
        ax.set_title(title)
        ax.set_yticks([])
        ax.set_xticks([])

    axD.cla();
    axD.plot( np.mean(np.abs(hidden_players[4][:lengths[1],:,1]-hidden_players[4][:lengths[0],:,0]),axis=1) );
    axD.set_xlim([0,lengths[0]]);
    
    display.clear_output(wait=True)
    display.display(fig)

In [35]:
help(axD.set_yticks)

Help on method set_yticks in module matplotlib.axes._base:

set_yticks(self, ticks, minor=False) method of matplotlib.axes._subplots.AxesSubplot instance
    Set the y ticks with list of *ticks*
    
    ACCEPTS: sequence of floats
    
    Keyword arguments:
    
      *minor*: [ *False* | *True* ]
        Sets the minor ticks if *True*

